In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [2]:
# File to Load 
loadfile= "../Resources/purchase_data.csv"

In [13]:
# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(loadfile)

In [14]:
# Display total number of players... 
totalplayers_df = purchase_data.drop(["Purchase ID","Age", "Gender", "Item ID", "Item Name", "Price"], axis=1)
cleantotalp_df = totalplayers_df.drop_duplicates(subset=["SN"])
playercount = cleantotalp_df.SN.value_counts()
playerscount = len(playercount)

# Create total players summary table
totalplayers_df = pd.DataFrame({ "Total Players" : [playerscount]})
totalplayers_df

,Total Players
0,576


In [15]:
# Find number of unique items, average price, number of purchases and total revenue

# Create summary table 
unique_items = purchase_data.drop_duplicates(subset=['Item Name'])
unique_item_count = len(unique_items)
avg_price = purchase_data["Price"].mean()
purchases = purchase_data["Purchase ID"].value_counts()
num_purchases = len(purchases) 
total_rev = purchase_data["Price"].sum()

# Summary table of items
summaryitems_df = pd.DataFrame({ "Number of Unique Items" : [unique_item_count],
                                 "Average Price" : "${:,.2f}".format(avg_price),
                                 "Nuber of Purchases": [num_purchases],
                                 "Total Revenue" : "${:,.2f}".format(total_rev)})
summaryitems_df

,Number of Unique Items,Average Price,Nuber of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [16]:
# Count & percentage of male/female players and colculate percentages of players

# create table with sn and gender, remove duplicates 
name_gender_df = purchase_data.drop(["Purchase ID","Age", "Item ID", "Item Name", "Price"], axis=1)
cleaned_ng_df = name_gender_df.drop_duplicates(subset=["SN"])
count_gender = cleaned_ng_df.Gender.value_counts()

# Divide gender count series by players count series for percentages
gender_percentage = count_gender.divide(playerscount, fill_value=0)*100

# Create gender summary table 
GenderDemo_df =  pd.DataFrame({ "Total Count": count_gender,
                                "Percent of Total": gender_percentage.map("{:.2f}%".format)})
GenderDemo_df

,Total Count,Percent of Total
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [17]:
# Purchase analysis of genders

# Count total purchases by gender, first group by gender, then perform analysis
group_gender = purchase_data.groupby(["Gender"])
purchase_count = group_gender["Purchase ID"].count()

# Average purchase by gender
avg_purchase = group_gender["Price"].mean()

# Average total purchases by gender
avg_purchase_total = group_gender["Price"].sum()

# Average purchase total by gender per player
avg_purchase_per = avg_purchase_total / count_gender

# Create data frame with obtained values 
gender_demographics_df = pd.DataFrame({ "Purchase Count": purchase_count, 
                                        "Average Purchase Price": avg_purchase,
                                        "Average Purchase Value":avg_purchase_total,
                                        "Avg Purchase Total per Person": avg_purchase_per})

# Provide index in top left as "Gender"
gender_demographics_df.index.name = "Gender"

# Format with currency style
gender_demographics_df.style.format({ "Average Purchase Value":"${:,.2f}",
                                      "Average Purchase Price":"${:,.2f}",
                                      "Avg Purchase Total per Person":"${:,.2f}"})

,Purchase Count,Average Purchase Price,Average Purchase Value,Avg Purchase Total per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [18]:
# Age Demographics

# Create bins for all ages 
age_bins = [0,9.90,14.90,19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

purchase_data["Age Group"] = pd.cut(purchase_data["Age"], age_bins, labels=group_names)
purchase_data

# New dataframe with "Age Group" added and grouped
grouped_age = purchase_data.groupby("Age Group")

# Count players by age category
total_count_age = grouped_age["SN"].nunique()

# Percentages by age category
age_percentage = (total_count_age/playerscount) * 100

# Create summary dataframe 
age_demo_df = pd.DataFrame({ "Total Count": total_count_age,
                             "Percentage of Players": age_percentage.map("{:.2f}%".format)})

# Remove imdex lable 
age_demo_df.index.name = None

age_demo_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [19]:
# Purchase analysis by age

# Count purchases by age group
age_purchase_count = grouped_age["Purchase ID"].count()

# Average purchase price by age 
avg_purchase_age = grouped_age["Price"].mean()

# Total purchases by age 
total_purchase_age = grouped_age["Price"].sum()

# Average purchase per person
avg_purchase_per_person = total_purchase_age / total_count_age

# Creat summary data frame 

age_purchase_summary_df = pd.DataFrame({ "Purchase Count" : age_purchase_count,
                                         "Average Purchase Price" : avg_purchase_age,
                                         "Total Purchase Value" : total_purchase_age,
                                         "Average Purchase Total per Person" : avg_purchase_per_person})

# Create index name 
age_purchase_summary_df.index.name = "Age Ranges"

# Format with currency style
age_purchase_summary_df.style.format({ "Average Purchase Price":"${:,.2f}",
                                       "Total Purchase Value":"${:,.2f}",
                                       "Average Purchase Total per Person":"${:,.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [20]:
# Top Spenders 

# Group by SN
spenders_data = purchase_data.groupby("SN")

# Count purchases per player
purchase_count_player = spenders_data["Purchase ID"].count()

# Average purchase price per player
avg_purchase_per_player = spenders_data["Price"].mean()

# Total purchase value 
total_player_value = spenders_data["Price"].sum()

# Create summary data frame 
spender_df = pd.DataFrame({ "Purchase Count" : purchase_count_player,
                            "Average Purchase Price" : avg_purchase_per_player,
                            "Total Purchase Value" : total_player_value})

# Sort table decending value
sorted_spender_df = spender_df.sort_values(["Total Purchase Value"], ascending = False).head()

# Add formatting for $
sorted_spender_df.style.format({ "Average Purchase Price":"${:,.2f}",
                                 "Total Purchase Value":"${:,.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [21]:
# Most popular items

# Create data frame with only item info 
items_df = purchase_data[["Item ID", "Item Name", "Price"]]

# Group by Item ID and Item Name
grouped_items = items_df.groupby(["Item ID", "Item Name"])

# Count total purchases of each item
item_purchase_count = grouped_items["Price"].count()

# Total purchase value
total_purchase_value = grouped_items["Price"].sum()

# Indicidual price per item
item_price = grouped_items["Price"].mean()

summary_items_df = pd.DataFrame({ "Purchase Count" : item_purchase_count,
                                  "Item Price" : item_price,
                                  "Total Purchase Value" : total_purchase_value})


# Sort data frame decending by purchase count
sorted_summary_items_df = summary_items_df.sort_values(["Purchase Count"], ascending=False).head()


# Format data frame
sorted_summary_items_df.style.format({ "Item Price":"${:,.2f}",
                                       "Total Purchase Value":"${:,.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [22]:
# Most profitable items

# Sort data frame decending by total purchase value
sorted_summary_items_df = summary_items_df.sort_values(["Total Purchase Value"], ascending=False).head()

# Format data frame
sorted_summary_items_df.style.format({ "Item Price":"${:,.2f}",
                                       "Total Purchase Value":"${:,.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
